<a href="https://colab.research.google.com/github/vgudur/machine-learning-learning-path-labs/blob/patch-3/naive-bayes/Vijji_Gudur_naive-bayes-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes Spam Filtering

### Overview

We all hate spam, so developing a classifier to classify email as spam or not spam is useful.  

### Builds on
None

### Run time
approx. 20-30 minutes

### Notes

We can do Naive Bayes classification.

## Step 1: Load Data
We will load the dataframe into pandas.  Since the outcome label is "ham" or "spam", that will be our label.

In [ ]:
import os
import urllib.request

data_location = 'SMSSpamCollection.txt'
data_url = 'https://elephantscale-public.s3.amazonaws.com/data/spam/SMSSpamCollection.txt'

if not os.path.exists(data_location):
    urllib.request.urlretrieve(data_url, data_location)
    print ("Downloading  ", data_url)
print ('data_location: ', data_location)

In [ ]:
import pandas as pd

dataset = pd.read_csv(data_location, sep='\t')
dataset

## Step 2 - Explore Data

In [ ]:
## TODO :  Count spam/ham
## Hint : group by 'isspam'
## Question : Is there a data skew?
dataset.groupby("???").size()

## Step 3: Vectorize Using TF/IDF

Let's use tf/idf for vecorization at first.  TF/IDF will take and count the instances of each term, and then divide by the total frequecy of that term in the entire dataset.  

This leads to very highly dimensional data, because every word in the document will lead to a dimension in the data.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


pipeline = Pipeline([('vec', CountVectorizer()),
                     ('tfidf', TfidfTransformer())])

x = pipeline.fit_transform(dataset['text'])
x

In [ ]:
## we can leave y as 'spam' / 'ham'
# y = dataset['isspam']

## TODO : encode y
## Hint : use the 'isspam' column
y = pd.factorize(dataset['???'])[0]
y

## Step 4: Split train/test

In [ ]:
## TODO: Use training / test split of 80%/20%

from sklearn.model_selection import train_test_split

x_train,x_test,y_train, y_test = train_test_split(x,y,  test_size=???)
## to control train/test split set random_state to a number
# x_train,x_test,y_train, y_test = train_test_split(x,y, random_state=0, test_size=0.3)

print ("x_train :" , x_train.shape )
print ("x_test :", x_test.shape)
print ("y_train :", y_train.shape)
print ("y_test :", y_test.shape)

## Step 5 : Run Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB ()

##TODO : fit on (x_train, y_train)


## TODO : Train and Evaluate the Model

- Compute training / testing accuracy
- compute confusion matrix

## Run your own data!

Now it's your turn!   Make a new dataframe with some sample test data of your own creation.  Make some "spammy" SMSes and some ordinary ones.  See how our spam filter does.

In [ ]:
mydata = pd.DataFrame ( { 'text' : [
                                     'can we meet for lunch?',
                                     'win win win instant tickets!',
                                     'ultra cheap medications!!!',
                                     'your text here'
]})

mydata

In [ ]:
my_x = pipeline.transform(mydata['text'])
my_x

In [ ]:
my_pred = model.predict(my_x)
my_pred

In [ ]:
mydata['prediction'] = my_pred
mydata